# Finance Team Assistant Agent with Classify & Exrtact

In this example, we use [LlamaExtract](https://developers.llamaindex.ai/python/cloud/llamaextract/getting_started/) and [LlamaClassify](https://developers.llamaindex.ai/python/cloud/llamaclassify/getting_started/), along with [Agent Workflows](https://developers.llamaindex.ai/python/llamaagents/workflows/) to build an intelligent agent that can triage incoming emails with attachments (like invoices or expenses) and respond accordingly.

This process consists of a few steps:
1. We want to classify incoming attachments: for this demo, we're classifying invoices and expenses
2. Next, based on what the result of the classification is, we want to extract some specific information: such as payee, due date for payment etc.
3. Finally, we want to take action accordingly. Here, we're simulating an email acknowledgement, and we're checking expenses against a budget.


## Setup

First, we need to install all the required packages and add the required API keys.

### Define Data Schemas and Extraction Agents

We'll define Pydantic models for the data we want to extract (Expense and Invoice) and then create LlamaExtract agents for each.

In [ ]:
!pip install llama-cloud-services llama-index-workflows llama-index-llms-openai

In [20]:
import os
from getpass import getpass

if os.getenv("LLAMA_CLOUD_API_KEY") is None:
  os.environ["LLAMA_CLOUD_API_KEY"] = getpass("Enter your LlamaCloud API Key")

if os.getenv("OPENAI_API_KEY") is None:
  os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API Key")

Enter your OpenAI API Key··········


## Create Extract Agents

In this scenario, we want to be able to simulate an inbox where employees or parnters can forward emails with attachments. These attachments could be invoices to be payed out, or internal expenses by employees that we need to check agains budgets etc.

Below, we create out `Expense` and `Invoice` schemas that we will use as the structure of extration agents.

In [ ]:
from llama_cloud_services import LlamaExtract
from pydantic import BaseModel, Field

class Expense(BaseModel):
  amount: float = Field(description="The amount of the expense")
  currency: str = Field(description="The currency of the expense")
  description: str = Field(description="A description of the expense")

class Invoice(BaseModel):
  amount: float = Field(description="The amount of the invoice")
  currency: str = Field(description="The currency of the invoice")
  due_date: str = Field(description="The due date of the invoice")
  payee: str = Field(description="The payee of the invoice")

llama_extract = LlamaExtract()
invoice_extract_agent = llama_extract.create_agent(name="Invoice_Extractor", data_schema=Invoice)
expense_extract_agent = llama_extract.create_agent(name="Expense_Extractor", data_schema=Expense)

## Build the Agent Workflow

Next, we define the custom events we want for our finance triage agent.

In [ ]:
from workflows.events import StartEvent, StopEvent, Event
from llama_index.llms.openai import OpenAI

class EmailReceived(StartEvent):
  sender: str
  subject: str
  body: str
  attachment: str

class ClassificationResult(Event):
  classification: str
  reason: str
  email: str
  attachment: str

class SendEmail(StopEvent):
  body: str

Our final `FinanceTeamAgent` has just 2 steps:
- `classify_email`: This is where we create a classifier with LlamaClassify, providinf rules for when an attacnhment is an invoice, vs when it's an expense
- `extract_contents`: Which is where we can design the next steps (in this case, we're simulating sending an appropriate email) depending on what the attachment has been classified as. We use our extract agents to extract the relevant information invoices or expenses.

In [83]:
from workflows import Workflow, step
from llama_cloud.types import ClassifierRule
from llama_cloud_services.beta.classifier.client import ClassifyClient
from llama_cloud.client import AsyncLlamaCloud



class FinanceTeamAgent(Workflow):
  def __init__(self, invoice_extract_agent, expense_extract_agent, *args, **kwargs):
    client = AsyncLlamaCloud(token=os.environ["LLAMA_CLOUD_API_KEY"])
    self.invoice_extract_agent = invoice_extract_agent
    self.expense_extract_agent = expense_extract_agent
    self.llm = OpenAI(model="gpt-5")
    self.classifier = ClassifyClient(client)
    super().__init__(*args, **kwargs)

  @step
  async def classify_email(self, ev: EmailReceived) -> ClassificationResult:
    rules = [
        ClassifierRule(
            type="invoice",
            description="This is an invoice for a contract that has to be payed out by the company. It may be forwarded from the partner or employee",
          ),
        ClassifierRule(
            type="expense",
            description="This is an expsense that's been submitted for a business trip that should be payed back to the employee in the next pay out cycle.",
            )]
    classification = await self.classifier.aclassify(files=ev.attachment, rules=rules)
    return ClassificationResult(classification=classification.items[0].result.type,
                                reason=classification.items[0].result.reasoning,
                                email=ev.sender,
                                attachment=ev.attachment)

  @step
  async def extract_contents(self, ev: ClassificationResult) -> SendEmail:
    if ev.classification == "expense":
      extracted_data = await self.expense_extract_agent.aextract(ev.attachment)
      if extracted_data.data["amount"] < 1000.0:
        body = self.llm.complete(f"""Construct an email acknowledging to {ev.email} that their expense of
                                    {extracted_data.data["amount"]} for {extracted_data.data["description"]} was accepted and will be payed back in the next payment cycle.""")
        return SendEmail(body=body.text)
      else:
        body = self.llm.complete(f"""Contruct an email the their expense of {extracted_data.data["amount"]} for {extracted_data.data["description"]} exceeds the
                                    budget so has been denied. Explain that they can reach out if this seems wrong""")
        return SendEmail(body=body.text)
    elif ev.classification == "invoice":
      extracted_data = await self.invoice_extract_agent.aextract(ev.attachment)
      body = self.llm.complete(f"""Construct a reply to {ev.email}, that the invoice has been received and gibe in for on who will
                                  be payed and how much based on the info in {extracted_data}""")
      return SendEmail(body=body.text)


In [84]:
agent = FinanceTeamAgent(invoice_extract_agent=invoice_extract_agent,
                         expense_extract_agent=expense_extract_agent,
                         timeout = 100.0)

## Try the Agent

To try thi agent, you can constuct an email below. Provide a file that could be an invoice or expense.

In [85]:
email = EmailReceived(sender = "tuana@runllama.ai",
                      subject = "Cowork Invoice",
                      body = "",
                      attachment = "/content/sb-receipt.png")

result = await agent.run(start_event = email)


Extracting files: 100%|██████████| 1/1 [00:05<00:00,  5.77s/it]


In [86]:
print(result.body)

To: tuana@runllama.ai
Subject: Expense Accepted – Starbucks Store #63225 (Mt. Juliet, TN) – $38.02

Hi Tuana,

We’ve reviewed your expense submission for $38.02 from Starbucks Store #63225 in Mt. Juliet, TN, and it has been accepted. The claim covers:
- One Venti Mocha Latte with oat milk
- One chocolate pie
- One grande white mocha with dat milk

Reimbursement will be included in the next payment cycle.

If you have any questions, please let us know.

Best regards,
[Your Name]
[Title]
[Company]
[Contact Information]
